In [1]:
%matplotlib inline
import numpy
import numpy as np
from matplotlib import pyplot
from amuse.plot import scatter
from amuse.units import (units, constants)
from amuse.ext.orbital_elements import new_binary_from_orbital_elements
from amuse.ext.orbital_elements import orbital_elements_from_binary
from amuse.lab import Particles
from amuse.lab import new_powerlaw_mass_distribution
from amuse.lab import nbody_system
from amuse.community.ph4.interface import ph4
from amuse.ext.LagrangianRadii import LagrangianRadii
def orbital_period(Mtot, a):
    return (((4 * numpy.pi**2) * a**3)/(constants.G * Mtot)).sqrt()

SMBH = Particles(1)
SMBH.name = "SMBH"
SMBH.mass = 4.154e6 | units.MSun
SMBH.position = (0, 0, 0) | units.pc
SMBH.velocity = (0, 0, 0) | units.kms

In [2]:
def relative_orbital_velocity(distance, mass=SMBH.mass):
    return (constants.G*mass/distance).sqrt()

def dist(body):
    return (body.x**2+body.y**2+body.z**2).sqrt()

def plot(body):
    r = 30
    scatter(body.x.value_in(units.pc), body.y.value_in(units.pc), s=5, alpha=0.5)
    #scatter(SMBH.x.value_in(units.pc), SMBH.z.value_in(units.pc), s=80, c='r')
    pyplot.xlim(-r, r)
    pyplot.ylim(-r, r)
    
def random_p(r_in=10, r_out=30):
    r=r_in + np.random.rand()*(r_out-r_in)
    a=np.random.rand()*2*np.pi
    x=r*np.cos(a) | units.pc
    y=r*np.sin(a) | units.pc
    z = 0 | units.pc
    return x,y,z
    
def random_v(distance):
    r = 30 | units.pc
    v=(((np.random.rand()+0.25)/1.25)*(2*(constants.G*SMBH.mass/distance-constants.G*SMBH.mass/(r)))).sqrt()
    a=np.random.rand()*2*np.pi
    vx=v*np.cos(a)
    vy=v*np.sin(a)
    vz=0 | units.kms
    return vx,vy,vz

In [3]:
x0 = np.array([])
y0 = np.array([])
x1 = np.array([])
y1 = np.array([])
x3 = np.array([])
y3 = np.array([])
for i in range(0, 1):
    bodies = Particles(2)
    bodies.name = ["BH1", "BH2"]
    bodies.mass = new_powerlaw_mass_distribution(2, 1.0|units.MSun, 100.0|units.MSun, 2.35)
    bodies[0].position = (random_p())
    bodies[1].position = (random_p())
    distance1, distance2 = dist(bodies[0]), dist(bodies[1])
    bodies[0].velocity = (random_v(distance1))
    bodies[1].velocity = (random_v(distance2))
    bodies.add_particles(SMBH)
    
    bodies1 = bodies.copy()
    BH3 = Particles(1)
    BH3.name = "BH3"
    BH3.mass = new_powerlaw_mass_distribution(1, 1.0|units.MSun, 100.0|units.MSun, 2.35)
    BH3.position = (random_p())
    BH3.velocity = (random_v(dist(BH3)))
    bodies1.add_particles(BH3)
    bodies1.move_to_center()
    bodies2 = bodies1.copy()
    np.save('bodies_original', bodies2)
    
    converter = nbody_system.nbody_to_si(bodies1.mass.sum(), bodies1.position.length())
#     bodies1.scale_to_standard(converter)
    gravity = ph4(converter, number_of_workers=32)
    gravity.particles.add_particles(bodies2)
    channel = gravity.particles.new_channel_to(bodies2)


    times = numpy.arange(0, 10000, 10) | units.Myr
    for time in times:
        gravity.evolve_model(time)
        channel.copy() # Copy from gravity.particles to bodies
        if not time.value_in(units.Myr)%10.0:
            print("Time=", time.in_(units.Myr))
        x0 = np.append(x0, bodies2[0].x.value_in(units.pc))
        y0 = np.append(y0, bodies2[0].y.value_in(units.pc))
        x1 = np.append(x1, bodies2[1].x.value_in(units.pc))
        y1 = np.append(y1, bodies2[1].y.value_in(units.pc))
        x3 = np.append(x3, bodies2[3].x.value_in(units.pc))
        y3 = np.append(y3, bodies2[3].y.value_in(units.pc))
        n_points = 10
        if len(x0)>n_points:
            pyplot.figure()
            pyplot.scatter(x0[-n_points:], y0[-n_points:], color="r", alpha=np.linspace(0,1,n_points))
            pyplot.scatter(x1[-n_points:], y1[-n_points:], color="b", alpha=np.linspace(0,1,n_points))
            pyplot.scatter(x3[-n_points:], y3[-n_points:], color="g", alpha=np.linspace(0,1,n_points))
            pyplot.plot(x0[-n_points:], y0[-n_points:], color="r", alpha=0.5, label=f"BH1 ({x0[-1]:.2e}, {y0[-1]:.2e})")
            pyplot.plot(x1[-n_points:], y1[-n_points:], color="b", alpha=0.5, label=f"BH2 ({x1[-1]:.2e}, {y1[-1]:.2e})")
            pyplot.plot(x3[-n_points:], y3[-n_points:], color="g", alpha=0.5, label=f"BH3 ({x3[-1]:.2e}, {y3[-1]:.2e})")
            pyplot.scatter(bodies1[2].x.value_in(units.pc), bodies1[2].y.value_in(units.pc), s=50, c="k")
            pyplot.xlim(-50, 50)
            pyplot.ylim(-50, 50)
            pyplot.legend(loc='best')
            pyplot.savefig('BH_trace.png')
            pyplot.close()
        
        b = (bodies2-SMBH).get_binaries(hardness=10)
        if(len(b)>0):
            print("Binary formed at", time)
            break
    
    pyplot.show()

/software/amuse/amuse-2021.6/lib/python3.9/site-packages/amuse/units/quantities.py:1161: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return numpy.array(self._number_list, dtype=self.unit.dtype)


Time= 0 Myr
Time= 10 Myr
Time= 20 Myr
Time= 30 Myr
Time= 40 Myr
Time= 50 Myr
Time= 60 Myr
Time= 70 Myr
Time= 80 Myr
Time= 90 Myr
Time= 100 Myr
Time= 110 Myr
Time= 120 Myr
Time= 130 Myr
Time= 140 Myr
Time= 150 Myr
Time= 160 Myr
Time= 170 Myr
Time= 180 Myr
Time= 190 Myr
Time= 200 Myr
Time= 210 Myr
Time= 220 Myr
Time= 230 Myr
Time= 240 Myr
Time= 250 Myr
Time= 260 Myr
Time= 270 Myr
Time= 280 Myr
Time= 290 Myr
Time= 300 Myr
Time= 310 Myr
Time= 320 Myr
Time= 330 Myr
Time= 340 Myr
Time= 350 Myr
Time= 360 Myr
Time= 370 Myr
Time= 380 Myr
Time= 390 Myr
Time= 400 Myr
Time= 410 Myr
Time= 420 Myr
Time= 430 Myr
Time= 440 Myr
Time= 450 Myr
Time= 460 Myr
Time= 470 Myr
Time= 480 Myr
Time= 490 Myr
Time= 500 Myr
Time= 510 Myr
Time= 520 Myr
Time= 530 Myr
Time= 540 Myr
Time= 550 Myr
Time= 560 Myr
Time= 570 Myr
Time= 580 Myr
Time= 590 Myr
Time= 600 Myr
Time= 610 Myr
Time= 620 Myr
Time= 630 Myr
Time= 640 Myr
Time= 650 Myr
Time= 660 Myr
Time= 670 Myr
Time= 680 Myr
Time= 690 Myr
Time= 700 Myr
Time= 710 Myr
Tim

Time= 5540 Myr
Time= 5550 Myr
Time= 5560 Myr
Time= 5570 Myr
Time= 5580 Myr
Time= 5590 Myr
Time= 5600 Myr
Time= 5610 Myr
Time= 5620 Myr
Time= 5630 Myr
Time= 5640 Myr
Time= 5650 Myr
Time= 5660 Myr
Time= 5670 Myr
Time= 5680 Myr
Time= 5690 Myr
Time= 5700 Myr
Time= 5710 Myr
Time= 5720 Myr
Time= 5730 Myr
Time= 5740 Myr
Time= 5750 Myr
Time= 5760 Myr
Time= 5770 Myr
Time= 5780 Myr
Time= 5790 Myr
Time= 5800 Myr
Time= 5810 Myr
Time= 5820 Myr
Time= 5830 Myr
Time= 5840 Myr
Time= 5850 Myr
Time= 5860 Myr
Time= 5870 Myr
Time= 5880 Myr
Time= 5890 Myr
Time= 5900 Myr
Time= 5910 Myr
Time= 5920 Myr
Time= 5930 Myr
Time= 5940 Myr
Time= 5950 Myr
Time= 5960 Myr
Time= 5970 Myr
Time= 5980 Myr
Time= 5990 Myr
Time= 6000 Myr
Time= 6010 Myr
Time= 6020 Myr
Time= 6030 Myr
Time= 6040 Myr
Time= 6050 Myr
Time= 6060 Myr
Time= 6070 Myr
Time= 6080 Myr
Time= 6090 Myr
Time= 6100 Myr
Time= 6110 Myr
Time= 6120 Myr
Time= 6130 Myr
Time= 6140 Myr
Time= 6150 Myr
Time= 6160 Myr
Time= 6170 Myr
Time= 6180 Myr
Time= 6190 Myr
Time= 6200

In [4]:
bodies3 = bodies2-SMBH-BH3
print(bodies3.potential_energy()+bodies3.kinetic_energy())

0.00122472949293 1.28198836793e+44 * m**2 * kg * s**-2


In [5]:
print(bodies1)
print(bodies2)

                 key         mass         name           vx           vy           vz            x            y            z
                   -         MSun         none  8028481.01748 * m * s**-1  8028481.01748 * m * s**-1          kms       parsec       parsec       parsec
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========
 6025014804053160861    9.519e+01          BH1    8.392e-04   -6.576e-04    0.000e+00   -2.454e+01   -2.549e+00    0.000e+00
 9678827015340460517    8.194e+01          BH2   -1.010e-03   -2.882e-03    0.000e+00   -1.652e+01   -2.612e+00    0.000e+00
15469067876880949035    4.154e+06         SMBH    6.837e-09    7.811e-08    0.000e+00    1.094e-03    5.083e-04    0.000e+00
12563622668057008491    6.384e+01          BH3   -3.997e-04   -4.028e-04    0.000e+00   -1.337e+01   -2.592e+01    0.000e+00
====================  ===========  ===========  ===========  ===========  ===========  ==========